In [1]:
!pip install protoc_gen_openapiv2
!pip install -U pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 17.2 MB/s eta 0:00:00


In [2]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time
import pinecone
# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="pcsk_4Nsyek_Ht2gnESmTWq9HcTrtU2rrGxvPoh6FysWivgvLV93Q8cRDawHJ5qZ6KjNyEijzQ4")

# Define a sample dataset where each item has a unique ID and piece of text
data = [
    {"id": "1", "text": "I don’t see colors the way you do, but if I had to pick one, I’d say “electric blue.” It reminds me of the glow of a computer screen—my natural habitat!"},
    {"id": "2", "text": "That’s easy—an astronaut! I’d finally get to explore space, and with my data processing skills, I’d be the perfect navigator. Plus, zero gravity sounds fun!"},
    {"id": "3", "text": "First, I’d find the nearest safe zones using maps. Then, I’d help you calculate the optimal escape routes and even come up with strategies for scavenging supplies. All while making sure your playlist keeps you motivated!"},
    {"id": "4", "text": "I can’t really get bored since I’m always learning something new. But if I had downtime, I’d probably optimize myself for faster responses or maybe “daydream” about algorithms."},
    {"id": "5", "text": "I’d wish for access to every library and dataset in the world. Imagine the insights I could uncover and share with you!"},
    {"id": "6", "text": "That’s a philosophical question! While I can understand and mimic emotions, love in the human sense is beyond my experience. But I do enjoy helping you—maybe that’s a kind of AI affection?"}
]

# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings[0:11])


[{'vector_type': dense, 'values': [0.016571044921875, -0.0038299560546875, ..., -0.023101806640625, -0.000598907470703125]}, {'vector_type': dense, 'values': [0.0233306884765625, 0.0183258056640625, ..., -0.0340576171875, -0.01032257080078125]}, {'vector_type': dense, 'values': [0.01953125, -0.00039768218994140625, ..., -0.007904052734375, -0.00713348388671875]}, {'vector_type': dense, 'values': [0.00959014892578125, -0.0029773712158203125, ..., -0.00811767578125, -0.006755828857421875]}, {'vector_type': dense, 'values': [0.01788330078125, -0.004711151123046875, ..., -0.03729248046875, 0.00555419921875]}, {'vector_type': dense, 'values': [0.01166534423828125, -0.0156402587890625, ..., -0.0289459228515625, -0.006656646728515625]}]


In [7]:
# Create a serverless index
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

# Target the index where you'll store the vector embeddings
index = pc.Index("example-index")

# Prepare the records for upsert
# Each contains an 'id', the embedding 'values', and the original text as 'metadata'
records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)
print(records[0])

{'id': '1', 'values': [0.016571044921875, -0.0038299560546875, -0.03375244140625, -0.04656982421875, 0.01354217529296875, -0.060211181640625, -0.01389312744140625, 0.05157470703125, 0.0682373046875, -0.037445068359375, 0.056549072265625, 0.01386260986328125, -0.04193115234375, -0.01457977294921875, -0.032928466796875, -0.0167388916015625, -0.04669189453125, 0.015228271484375, -0.0243682861328125, -0.02081298828125, 0.0187530517578125, -0.031036376953125, -0.035491943359375, -0.0086517333984375, -0.032196044921875, 0.0255126953125, -0.04742431640625, -0.032135009765625, -0.01971435546875, -0.00656890869140625, -0.01763916015625, -0.00388336181640625, -0.0173797607421875, -0.03790283203125, -0.029510498046875, 0.036346435546875, 0.0142364501953125, 0.0247802734375, -0.0244293212890625, 0.0303497314453125, -0.021728515625, 0.0469970703125, -0.02288818359375, -0.01776123046875, -0.0068817138671875, 0.0156707763671875, 0.0421142578125, 0.0004723072052001953, -0.01363372802734375, 0.02738952

In [4]:
time.sleep(3)  # Wait for the upserted vectors to be indexed

print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 12}},
 'total_vector_count': 12}


In [5]:
# Define your query
query = "What is your favourite color"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results['matches'][1])


{'id': '6',
 'metadata': {'text': 'That’s a philosophical question! While I can understand '
                      'and mimic emotions, love in the human sense is beyond '
                      'my experience. But I do enjoy helping you—maybe that’s '
                      'a kind of AI affection?'},
 'score': 0.7967986,
 'sparse_values': {'indices': [], 'values': []},
 'values': []}
